In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from typing import Sequence

from superprimitive_fusion.scanner import (
    virtual_scan,
    mesh_depth_image,
    virtual_mesh_scan,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
)

In [ ]:
names = (
    ('mustard-bottle', 'mustard-bottle.obj'),
    ('table', 'table.obj'),
    # ('power-drill', 'power-drill.obj'),
)

meshes = dict()
for foldername,filename in names:
    print(f'Getting the {filename.split('.')[0]}')
    
    mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(mesh)
    mesh.compute_vertex_normals()

    meshes[foldername] = mesh

meshlist = list(meshes.values())

Getting the mustard-bottle
Getting the table


In [3]:
centres = []
for meshname, mesh in meshes.items():
    if meshname == 'table':
        continue
    centres.append(mesh.get_center())

centres = np.vstack(centres)

obj_centre = centres.mean(axis=0)

In [4]:
mesh: o3d.geometry.TriangleMesh = meshlist
cam_centre=polar2cartesian(1.0, 30, 90)
look_dir=obj_centre
width_px: int = 360
height_px: int = 240
fov: float = 70.0
k: float = 3.5
max_normal_angle_deg = None

In [5]:
result = virtual_scan(
    meshlist,
    cam_centre,
    look_dir,
    width_px=width_px,
    height_px=height_px,
    fov=fov,
)

In [ ]:
def moge_like_vertex_noise(
    verts: np.ndarray,
    cam_centre: np.ndarray | tuple,
    look_dir: np.ndarray | tuple,
    normals: np.ndarray | None = None,
    *,
    # --- depth/aniso noise ---
    depth_sigma_linear: float = 0.01,    # ~1% of depth along the viewing ray
    depth_sigma_quadratic: float = 0.0,  # optional extra growth with depth^2
    lateral_frac: float = 0.3,           # lateral noise as a fraction of ray noise
    grazing_boost: float = 1.0,          # extra noise at grazing angles (0=off)
    # --- low-freq bias (piecewise warping) ---
    bias_amp: float = 0.02,              # ~2% of depth, deterministic sinusoid
    bias_freq: float = 0.5,              # cycles per scene unit (tune to taste)
    # --- outliers / dropouts ---
    outlier_prob: float = 0.003,         # ~0.3% heavy-tail blips along the ray
    outlier_scale: float = 0.10,         # ~10% of depth for those blips
    dropout_prob: float = 0.0,           # set some vertices to NaN (0 to disable)
    # --- optional global MoGe-like ambiguity ---
    global_scale_std: float = 0.0,       # e.g., 0.05 for +-5% scale jitter
    global_shift_std: float = 0.0,       # std (scene units) for 3D shift jitter
    seed: int | None = None,
) -> np.ndarray:
    """
    Add MoGe-style monocular geometry noise to a vertex array (N,3).
    - Keeps vertex order unchanged.
    - Returns a new (N,3) array; input is not modified.
    - If normals are provided (N,3), noise increases near grazing angles.

    Parameters marked with * are the ones you'll typically tweak.
    Assumes verts & cam are in the same metric (e.g., meters).
    """
    rng = np.random.default_rng(seed)

    V = np.asarray(verts, dtype=np.float32)
    C = np.asarray(cam_centre, dtype=np.float32).reshape(1, 3)
    L = np.asarray(look_dir, dtype=np.float32).reshape(3)
    L /= (np.linalg.norm(L) + 1e-12)

    # Vector from cam to vertex, per-vertex viewing direction & depth along global look_dir
    R = V - C
    d = R @ L                  # signed depth along camera look axis
    d_pos = np.maximum(d, 0.0) # avoid negative depths (behind camera)

    # Per-vertex ray direction (camera -> point)
    R_norm = np.linalg.norm(R, axis=1, keepdims=True)
    ray_dir = np.divide(R, np.maximum(R_norm, 1e-12))

    # Orthonormal lateral basis (u, v) for each ray
    # Pick a helper axis that isn't collinear with ray_dir
    helper = np.array([0.0, 0.0, 1.0], dtype=np.float32)
    collinear = (np.abs(ray_dir @ helper) > 0.9).ravel()
    helper = np.where(collinear[:, None], np.array([0.0, 1.0, 0.0], dtype=np.float32), helper)
    u = np.cross(ray_dir, helper); u /= (np.linalg.norm(u, axis=1, keepdims=True) + 1e-12)
    v = np.cross(ray_dir, u)

    # Base anisotropic sigmas (depth grows noise mostly along the ray)
    sigma_ray = depth_sigma_linear * d_pos + depth_sigma_quadratic * (d_pos ** 2)

    # Extra noise at grazing angles using normals if available
    if normals is not None:
        N = np.asarray(normals, dtype=np.float32)
        N /= (np.linalg.norm(N, axis=1, keepdims=True) + 1e-12)
        # cos(theta) between surface normal and incoming view (−ray_dir)
        cos_th = np.abs(np.sum(N * (-ray_dir), axis=1))
        boost = 1.0 + grazing_boost * (1.0 - cos_th)  # bigger when grazing (cos_th ~ 0)
        sigma_ray = sigma_ray * boost

    sigma_lat = lateral_frac * sigma_ray

    # Low-frequency, depth-scaled bias (deterministic warping)
    k = rng.normal(size=3); k /= (np.linalg.norm(k) + 1e-12)
    phase = rng.uniform(0, 2*np.pi)
    bias = bias_amp * d_pos * np.sin((V @ (k * (2*np.pi*bias_freq))).ravel() + phase)

    # Sample Gaussian noise components
    n_ray = rng.normal(0.0, sigma_ray)
    n_u   = rng.normal(0.0, sigma_lat)
    n_v   = rng.normal(0.0, sigma_lat)

    # Add deterministic bias along the ray
    n_ray = n_ray + bias

    disp = (n_ray[:, None] * ray_dir) + (n_u[:, None] * u) + (n_v[:, None] * v)

    # Rare heavy-tailed outliers along the ray
    if outlier_prob > 0:
        m = rng.random(len(V)) < outlier_prob
        if np.any(m):
            # Laplace heavy-tail offset proportional to current depth
            o = rng.laplace(0.0, outlier_scale * np.maximum(d_pos[m], 1e-6))
            disp[m] += (o[:, None] * ray_dir[m])

    Vn = V + disp

    # Optional global affine ambiguity (MoGe predicts affine-invariant point maps)
    # Apply as a similarity transform around the camera centre, plus 3D shift.
    if (global_scale_std > 0) or (global_shift_std > 0):
        s = rng.normal(1.0, global_scale_std)
        t = rng.normal(0.0, global_shift_std, size=3)
        Vn = C + s * (Vn - C) + t

    # Optional dropouts (e.g., infinity/sky masks); set to NaN to mark invalid.
    if dropout_prob > 0:
        drop = rng.random(len(Vn)) < dropout_prob
        Vn = Vn.copy()
        Vn[drop] = np.nan

    return Vn


In [ ]:
for name, value in result.items():
    print(f'{name}:\ttype: {type(value)}\tshape: {value.shape}')
    
"""
verts:	type: <class 'numpy.ndarray'>	shape: (240, 360, 3)
vcols:	type: <class 'numpy.ndarray'>	shape: (240, 360, 3)
norms:	type: <class 'numpy.ndarray'>	shape: (240, 360, 3)
segmt:	type: <class 'numpy.ndarray'>	shape: (240, 360)
"""

verts:	type: <class 'numpy.ndarray'>	shape: (240, 360, 3)
vcols:	type: <class 'numpy.ndarray'>	shape: (240, 360, 3)
norms:	type: <class 'numpy.ndarray'>	shape: (240, 360, 3)
segmt:	type: <class 'numpy.ndarray'>	shape: (240, 360)


In [12]:
verts = result['verts'].reshape(-1, 3)
verts = verts[np.any(np.isfinite(verts), axis=1)]

norms = result['norms'].reshape(-1, 3)
norms = norms[np.any(np.isfinite(result['verts'].reshape(-1, 3)), axis=1)]

verts_noised = moge_like_vertex_noise(
    verts=verts,
    cam_centre=cam_centre,
    look_dir=look_dir,
    normals=norms,
    # --- depth/aniso noise ---
    depth_sigma_linear = 0.5,    # ~1% of depth along the viewing ray
    depth_sigma_quadratic = 0.05,  # optional extra growth with depth^2
    lateral_frac = 0.3,           # lateral noise as a fraction of ray noise
    grazing_boost = 1.0,          # extra noise at grazing angles (0=off)
    # --- low-freq bias (piecewise warping) ---
    bias_amp = 0.02,              # ~2% of depth, deterministic sinusoid
    bias_freq = 0.5,              # cycles per scene unit (tune to taste)
    # --- outliers / dropouts ---
    outlier_prob = 0.003,         # ~0.3% heavy-tail blips along the ray
    outlier_scale = 0.10,         # ~10% of depth for those blips
    dropout_prob = 0.0,           # set some vertices to NaN (0 to disable)
    # --- optional global MoGe-like ambiguity ---
    global_scale_std = 0.0,       # e.g., 0.05 for ±5% scale jitter
    global_shift_std = 0.0,       # std (scene units) for 3D shift jitter
    seed = None,
)

In [13]:
pcd1 = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(verts))
pcd1.paint_uniform_color(np.array([1, 0, 0]))
pcd2 = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(verts_noised))
pcd2.paint_uniform_color(np.array([0, 1, 0]))

o3d.visualization.draw_geometries([pcd1, pcd2])